In [ ]:
from sec_edgar_downloader import Downloader
#needs to comply with sec privacy :(( )
dl = Downloader("test", "test@test.com")
import pandas as pd 
import time
import requests

#sec offers an api to get the daily or quartely indexes, metadata for all fillings
#there is no endpoint to specifically search by form type
#the actual data can only be downloaded with an accesion number and cik
#using cik fetched from daily index, fetch all fillings by company and select those with type D 
#download the actual file using url from the data line(the current data url returns the entire txt, we only need the actual xml file for the filing ) 
## Tom: added inputs
## Tom: Maybe add this lateremail = input("Enter your email for SEC requests (e.g., example@example.com): ")?
## Tom: It would be also nice to have the quarter variable derived from the other inputs. 
# //todo, save files to not fetch every time 
year = int(input("Enter the year in which you are interested (e.g., 2024). "))
quarter = int(input("Enter the quarter in which you are interested (e.g. 4). "))
month = int(input("Enter the month of interest in numerical format (e.g. 11)."))
day = int(input("Enter the day of interest in numerical format (e.g. 27)."))
date = f"{year}{month}{day}"
base_url = 'https://www.sec.gov/Archives/edgar/daily-index'
index_url = f'{base_url}/{year}/QTR{quarter}/company.{date}.idx'

base_full_index_url ='https://www.sec.gov/Archives/edgar/full-index'
full_index_url =  f'{base_full_index_url}/2024/QTR3/company.idx'

headers = {
    'User-Agent': 'Test (test@test.com)',
    'Accept-Encoding': 'gzip, deflate',
    'Host': 'www.sec.gov',
    'Connection': 'keep-alive',
}
response = requests.get(index_url, headers=headers)
#response = requests.get(full_index_url, headers=headers)

if response.status_code == 403:
    print("Access denied. Ensure you have a proper User-Agent header.")
    exit()

response.raise_for_status() 


content = response.text

print(content)
lines = content.splitlines()


form_d_filings = []



header = ["Company Name", "Form Type", "CIK", "Date Filed", "File Name"]
#data lines skip 3 for daily index, 7 for full index 
#daily index returns idx file with all the fillings for the day, fixed width format, parse it into pandas 
records = [] 
#daily index formatting 

data_lines= lines[3:]
for line in data_lines:
    if line.strip():  
        company_name = line[:60].strip()
        form_type = line[60:71].strip()
        cik = line[71:82].strip()
        date_filed = line[82:92].strip()
        file_name = line[92:].strip()
        records.append([company_name, form_type, cik, date_filed, file_name])


#quarter index formatting 
'''
data_lines=lines[8:]
for line in data_lines:
    if line.strip():  # Ignore empty lines
        company_name = line[0:59].strip()
        form_type = line[59:72].strip()
        cik = line[72:87].strip()
        date_filed = line[87:102].strip()
        file_name = line[102:].strip()
        
        records.append([company_name, form_type, cik, date_filed, file_name])
        '''
print(records[1])
df = pd.DataFrame(records, columns=header)
print(response.text)
print(len(records))

form_d_df = df[df["Form Type"] == "D"] 
row_dict = form_d_df.iloc[1].to_dict()  
print("Row at index 1 as a dictionary:")
for key, value in row_dict.items():
    print(f"{key}: {value}")
#actual data can be downloaded with CIK and an accession number for the filling
form_d_df['Acession_number'] = form_d_df['File Name'].str.split('/').str[-1].str.replace('-', '').str.replace('.txt', '', regex=False)


base_url = "https://www.sec.gov/Archives/edgar/data/"
headers = {
    'User-Agent': 'Test (test@test.com)',
    'Accept-Encoding': 'gzip, deflate',
    'Host': 'www.sec.gov',
    'Connection': 'keep-alive',
}

xml_data_list = []
for index, row in form_d_df.iterrows():
    print("working:" + row['CIK'])
    cik = row['CIK']
    accession_number = row['Acession_number']
    url = f"{base_url}{cik}/{accession_number}/primary_doc.xml"
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            xml_data_list.append(response.text)
        else:
            xml_data_list.append(f"Failed for {url} with status code {response.status_code}")
    except Exception as e:
        xml_data_list.append(f"Error for {url}: {str(e)}")
print(len(form_d_df))

#//todo parse xml file to pandas 

In [ ]:
# Parsing the XML file to extract the relevant information

import xml.etree.ElementTree as ET
import  plotly.express as px
import plotly.graph_objects as go
# Definine parsing function
def xml_parse(root):
    def extract_text(element, path):
        found = element.find(path)
        return found.text if found is not None else None

    # Parse primary issuer data
    primary_issuer = root.find(".//primaryIssuer")
    if primary_issuer is None:
        return None
    data_primary_issuer = {
        "Central Index Key": extract_text(primary_issuer, ".//cik"),
        "Name of the Entity": extract_text(primary_issuer, ".//entityName"),
        "Address": extract_text(primary_issuer, ".//street1"),
        "Address - Specification": extract_text(primary_issuer, ".//street2"),
        "City": extract_text(primary_issuer, ".//city"),
        "State or Country - Physical Location": extract_text(primary_issuer, ".//stateOrCountry"),
        "State or Country - Full - Physical Location": extract_text(primary_issuer, ".//stateOrCountryDescription"),
        "Zip Code / Postal Code": extract_text(primary_issuer, ".//zipCode"),
        "Issuer Phone Number": extract_text(primary_issuer, ".//issuerPhoneNumber"),
        "State or Country - Legal Jurisdiction": extract_text(primary_issuer, ".//jurisdictionOfInc"),
        "Previous Names Used by the Issuer": extract_text(primary_issuer, ".//issuerPreviousNameList/value"),
        "Previous Names from EDGAR": extract_text(primary_issuer, ".//edgarPreviousNameList/value"),
        "Type of the Entity": extract_text(primary_issuer, ".//entityType"),
        "Type of the Entity - Specification": extract_text(primary_issuer, ".//entityTypeOtherDesc"),
    }

    # Parse related persons data
    related_persons = root.findall(".//relatedPersonsList/relatedPersonInfo")
    data_related_persons = {}
    for i, related_person in enumerate(related_persons, start=1):
        data_related_persons[f"First Name - Person {i}"] = extract_text(related_person, ".//relatedPersonName/firstName"),
        data_related_persons[f"Last Name - Person {i}"] = extract_text(related_person, ".//relatedPersonName/lastName"),
        data_related_persons[f"Associated Address - Person {i}"] = extract_text(related_person, ".//relatedPersonAddress/street1"),
        data_related_persons[f"Associated Address - Specification - Person {i}"] = extract_text(related_person, ".//relatedPersonAddress/street2"),
        data_related_persons[f"Associated City - Person {i}"] = extract_text(related_person, ".//relatedPersonAddress/city"),
        data_related_persons[f"Associated State or Country - Person {i}"] = extract_text(related_person, ".//relatedPersonAddress/stateOrCountry"),
        data_related_persons[f"Associated State or Country - Full - Person {i}"] = extract_text(related_person, ".//relatedPersonAddress/stateOrCountryDescription"),
        data_related_persons[f"Associated Zip Code - Person {i}"] = extract_text(related_person, ".//relatedPersonAddress/zipCode"),

        relationships = related_person.findall(".//relatedPersonRelationshipList/relationship")
        for j, relationship in enumerate(relationships, start=1):
            data_related_persons[f"Relationship {j} with the Entity - Person {i}"] = relationship.text if relationship is not None else None

        data_related_persons[f"Clarification of the relationship - Person {i}"] = extract_text(related_person, ".//relationshipClarification"),

    # Parse offering data
    offering_data = root.find(".//offeringData")
    data_offering_data = {
        "Industry": extract_text(offering_data, ".//industryGroup/industryGroupType"),
        "Revenue Range of the Entity": extract_text(offering_data, ".//issuerSize/revenueRange"),
        "Federal Exemption or Exclusions Claimed by the Entity": ", ".join([item.text for item in offering_data.findall(".//federalExemptionsExclusions/item") if item is not None]) if offering_data is not None else None,
        "Is the Entry an Amendment to Another Filing?": extract_text(offering_data, ".//typeOfFiling/newOrAmendment/isAmendment"),
        "Date of the First Sale Under this Filing": extract_text(offering_data, ".//typeOfFiling/dateOfFirstSale/value"),
        "Is the Offering Duration Intended to be More Than a Year?": extract_text(offering_data, ".//durationOfOffering/moreThanOneYear"),
        "Is the Entity Offering Equity": extract_text(offering_data, ".//typesOfSecuritiesOffered/isEquityType"),
        "Is the Entity Creating or Managing a Pooled Fund": extract_text(offering_data, ".//typesOfSecuritiesOffered/isPooledInvestmentFundType"),
        "Is the Offering Part of a Business Combination Transaction?": extract_text(offering_data, ".//isPooledInvestmentFundType/isBusinessCombinationTransaction"),
        "Is the Offering Part of a Business Combination Transaction? - Clarification": extract_text(offering_data, ".//businessCombinationTransaction/clarificationOfResponse"),
        "Minimum Investment Amount Accepted": extract_text(offering_data, ".//minimumInvestmentAccepted"),
        #"Entities Compensated for Selling the Securities": extract_text(offering_data, ".//salesCompensationList"),
        # The line above is commented out because we don't see much use in it
        "Total Amount Offered": extract_text(offering_data, ".//offeringSalesAmounts/totalOfferingAmount"),
        "Total Amount Sold So Far": extract_text(offering_data, ".//offeringSalesAmounts/totalAmountSold"),
        "Total Amount Remaining to be sold": extract_text(offering_data, ".//offeringSalesAmounts/totalRemaining"),
        "Clarifications Regarding the Offering and Sales amounts": extract_text(offering_data, ".//offeringSalesAmounts/clarificationOfResponse"),
        "Are Non-accredited Investors Participating in the Offering?": extract_text(offering_data, ".//investors/hasNonAccreditedInvestors"),
        "Number of Investors Who Already Invested Under Current Offering": extract_text(offering_data, ".//investors/totalNumberAlreadyInvested"),
        "Commision Amount Paid for Sales": extract_text(offering_data, ".//salesCommissionsFindersFees/salesCommissions/dollarAmount"),
        "Finder's Fees Amount Paid": extract_text(offering_data, ".//salesCommissionsFindersFees/findersFees/dollarAmount"),
        "Clarification Regarding the Finder's Fees Amount Paid": extract_text(offering_data, ".//salesCommissionsFindersFees/clarificationOfResponse"),
        "Amount of the Raised Capital Already Used for a Purpose": extract_text(offering_data, ".//useOfProceeds/grossProceedsUsed/dollarAmount"),
        "Clarification Regarding the Amount of the Raised Capital Already Used for a Purpose": extract_text(offering_data, ".//useOfProceeds/clarificationOfResponse"),
        "Name of the Entity Representative Signing the Filing": extract_text(offering_data, ".//signatureBlock/authorizedRepresentative"),
        "Name of the Issuer Entity": extract_text(offering_data, ".//signatureBlock/signature/issuerName"),
        "Name of the Person Signing": extract_text(offering_data, ".//signatureBlock/signature/signatureName"),
        "Name of the Person Signing - Affirmation": extract_text(offering_data, ".//signatureBlock/signature/nameOfSigner"),
        "Title of the Person Signing": extract_text(offering_data, ".//signatureBlock/signature/signatureTitle"),
        "Date the Form Was Signed": extract_text(offering_data, ".//signatureBlock/signature/signatureDate")
    }
    # Combine all data
    data = {**data_primary_issuer, **data_related_persons, **data_offering_data}
    return data

data_fin = []

for information in xml_data_list:
    print(information)
    root = ET.fromstring(information)
    data_fin.append(xml_parse(root))

df_main = pd.DataFrame(data_fin)
#pd.set_option('display.max_columns', 500)
# A very inefficient way to reorganize the columns
#df_main.reindex((["Central Index Key", "Name of the Entity", "Address", "Address - Specification", "City", "State or Country - Physical Location", "State or Country - Full - Physical Location", "Zip Code / Postal Code", "Issuer Phone Number", "State or Country - Legal Jurisdiction", "Previous Names Used by the Issuer", "Previous Names from EDGAR", "Type of the Entity", "Type of the Entity - Specification", "Industry", "Revenue Range of the Entity", "Federal Exemption or Exclusions Claimed by the Entity", "Is the Entry an Amendment to Another Filing?", "Date of the First Sale Under this Filing", "Is the Offering Duration Intended to be More Than a Year?", "Is the Entity Offering Equity", "Is the Entity Creating or Managing a Pooled Fund", "Is the Offering Part of a Business Combination Transaction?", "Is the Offering Part of a Business Combination Transaction? - Clarification", "Minimum Investment Amount Accepted", "Entities Compensated for Selling the Securities", "Total Amount Offered", "Total Amount Sold So Far", "Total Amount Remaining to be sold", "Clarifications Regarding the Offering and Sales amounts", "Are Non-accredited Investors Participating in the Offering?", "Number of Investors Who Already Invested Under Current Offering", "Commision Amount Paid for Sales", "Finder's Fees Amount Paid", "Clarification Regarding the Finder's Fees Amount Paid", "Amount of the Raised Capital Already Used for a Purpose", "Clarification Regarding the Amount of the Raised Capital Already Used for a Purpose", "Name of the Entity Representative Signing the Filing", "Name of the Issuer Entity", "Name of the Person Signing", "Name of the Person Signing - Affirmation", "Title of the Person Signing", "Date the Form Was Signed", "First Name - Person 1", "Last Name - Person 1", "Associated Address - Person 1", "Associated Address - Specification - Person 1", "Associated City - Person 1", "Associated State or Country - Person 1", "Associated State or Country - Full - Person 1", "Associated Zip Code - Person 1", "Relationship 1 with the Entity - Person 1", "Relationship 2 with the Entity - Person 1", "Relationship 3 with the Entity - Person 1", "Clarification of the relationship - Person 1", "First Name - Person 2", "Last Name - Person 2", "Associated Address - Person 2", "Associated Address - Specification - Person 2", "Associated City - Person 2", "Associated State or Country - Person 2", "Associated State or Country - Full - Person 2", "Associated Zip Code - Person 2", "Relationship 1 with the Entity - Person 2", "Relationship 2 with the Entity - Person 2", "Clarification of the relationship - Person 2", "First Name - Person 3", "Last Name - Person 3", "Associated Address - Person 3", "Associated Address - Specification - Person 3", "Associated City - Person 3", "Associated State or Country - Person 3", "Associated State or Country - Full - Person 3", "Associated Zip Code - Person 3", "Relationship 1 with the Entity - Person 3", "Relationship 2 with the Entity - Person 3", "Clarification of the relationship - Person 3", "First Name - Person 4", "Last Name - Person 4", "Associated Address - Person 4", "Associated Address - Specification - Person 4", "Associated City - Person 4", "Associated State or Country - Person 4", "Associated State or Country - Full - Person 4", "Associated Zip Code - Person 4", "Relationship 1 with the Entity - Person 4", "Clarification of the relationship - Person 4", "First Name - Person 5", "Last Name - Person 5", "Associated Address - Person 5", "Associated Address - Specification - Person 5", "Associated City - Person 5", "Associated State or Country - Person 5", "Associated State or Country - Full - Person 5", "Associated Zip Code - Person 5", "Relationship 1 with the Entity - Person 5", "Clarification of the relationship - Person 5"]), axis=1)

# Cleaning the data
df_main = df_main.fillna("N/A")
for column in df_main.columns:
    df_main[column] = df_main[column].apply(lambda x: x[0] if isinstance(x, tuple) else x)

df_main = df_main.replace("None", "N/A")

redundant_strings = ["/s/", "/bem/", "c/o", "n/a"]
for x in redundant_strings:
    df_main = df_main.replace(x, "", regex=True)

df_main = df_main.replace("", "N/A")
df_main = df_main.fillna("N/A")





In [ ]:
all_states = [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
]
state_full_names = [
    "ALABAMA", "ALASKA", "ARIZONA", "ARKANSAS", "CALIFORNIA", "COLORADO", "CONNECTICUT", 
    "DELAWARE", "FLORIDA", "GEORGIA", "HAWAII", "IDAHO", "ILLINOIS", "INDIANA", "IOWA", 
    "KANSAS", "KENTUCKY", "LOUISIANA", "MAINE", "MARYLAND", "MASSACHUSETTS", "MICHIGAN", 
    "MINNESOTA", "MISSISSIPPI", "MISSOURI", "MONTANA", "NEBRASKA", "NEVADA", "NEW HAMPSHIRE", 
    "NEW JERSEY", "NEW MEXICO", "NEW YORK", "NORTH CAROLINA", "NORTH DAKOTA", "OHIO", 
    "OKLAHOMA", "OREGON", "PENNSYLVANIA", "RHODE ISLAND", "SOUTH CAROLINA", "SOUTH DAKOTA", 
    "TENNESSEE", "TEXAS", "UTAH", "VERMONT", "VIRGINIA", "WASHINGTON", "WEST VIRGINIA", 
    "WISCONSIN", "WYOMING"
]
df =  pd.read_csv("2024_QTR4.csv")

state_name_to_abbreviation = dict(zip(state_full_names, all_states))

jurisdiction_counts = (
    df["State or Country - Legal Jurisdiction"]
    .map(state_name_to_abbreviation)
    .value_counts()  
    .reset_index()
)
jurisdiction_counts.columns = ["State", "JurisdictionCount"]
state_counts_simple = (
    df["State or Country - Physical Location"]
    .value_counts()
    .reset_index()
)
state_counts_simple.columns = ["State", "TotalCount"]
state_counts_simple["TotalCount"] = state_counts_simple["TotalCount"].fillna(0).astype(int)
print(state_counts_simple)

merged_counts = pd.merge(
    jurisdiction_counts, state_counts_simple, on="State", how="outer"
).fillna(0)

merged_counts["Percentage"] = (
    (merged_counts["JurisdictionCount"] / merged_counts["TotalCount"]) * 100
).fillna(0)

# Ensure all states are represented in the DataFrame with default count 0
#state_counts_simple = df["State or Country - Physical Location"].value_counts().reset_index()
#state_counts_simple.columns = ["State", "Count"]
#state_counts_simple["Count"] = state_counts_simple["Count"].fillna(0).astype(int)

merged_counts = merged_counts[["State", "Percentage"]]
print(merged_counts.head(55))

fig_simple = px.choropleth(
    merged_counts,
    locations="State",
    locationmode="USA-states",
    color="Percentage",
    range_color=(0,100),
    scope="usa",
    color_continuous_scale="Viridis",
    title="Count of D type fillings by US state"
)

# Displaying the map
fig_simple.show()

In [1]:
import pandas as pd 
import  plotly.express as px
import plotly.graph_objects as go
import zipfile
zip_file_path = "data.zip"
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall("extracted_data") 


csv_file_path = "extracted_data/2024_QTR1.csv"
df = pd.read_csv(csv_file_path)
total_income_by_state = (
    df.groupby(df["State or Country - Physical Location"])["Total Amount Sold So Far"]
    .sum()
    .reset_index()
)

total_income_by_state.columns = ["State", "TotalIncome" ]

# Create the choropleth map
fig_income = px.choropleth(
    total_income_by_state,
    locations="State",
    locationmode="USA-states",
    color="TotalIncome",
    scope="usa",
    color_continuous_scale="Viridis",
    title="Total Income Raised by State",
    labels={"TotalIncome": "Total Income ($)"}
)

fig_income.show()

/var/folders/d2/m57xcgf11tb1brc3mk8jsgjm0000gn/T/ipykernel_43521/744353279.py:11: DtypeWarning: Columns (89,100,112,181,191,201,211,221,224,225,226,227,228,229,230,232,233,234,235,236,237,238,239,240,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,262,263,264,265,266,267,268,269,270,272,273,274,275,277,278,280,281,282,283,284,285,286,288,289,290,291,292,293,294,295,296,298,299,300,301,302,303,304,305,306,308,309,310,311,312,313,314,315,316,318,320,321,322,323,324,325,326,328,330,331,332,333,334,335,336,338,340,341,342,343,344,345,346,348,350,351,352,353,354,355,356,358,360,361,362,363,364,365,367,368,369,371,373,374,375,377,378,379,381,383,384,385,387,388,389,391,393,394,395,397,398,399,401,403,404,405,407,408,409,411,413,414,415,417,418,419,421,423,424,425,427,428,429,431,433,434,435,437,438,439,441,443,444,445,447,448,449,451,453,454,455,457,458,459,461,463,464,465,467,468,469,471,473,474,475,477,478,479,481,483,484,485,487,488,489,491,493,494,495,497,498,

In [ ]:
print(len(xml_data_list))

In [ ]:
df_main.iloc[8,25]